In [2]:
# %pip install --upgrade openai
# %pip show openai

Name: openai
Version: 2.6.1
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: c:\Users\amenjw1\.conda\envs\jared_studio\Lib\site-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [1]:
from openai import OpenAI
from openai.types.fine_tuning import ReinforcementMethod, ReinforcementHyperparameters, PythonGrader

client = OpenAI()

ImportError: cannot import name 'PythonGrader' from 'openai.types.fine_tuning' (c:\Users\amenjw1\.conda\envs\jared_studio\Lib\site-packages\openai\types\fine_tuning\__init__.py)

In [5]:
client.files.create(
  file=open("RFT_val_test.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-1uXN8NvBU5QEKj9nT4omVC', bytes=1105, created_at=1761521524, filename='RFT_val_test.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)

In [2]:
# For RFT
# prompt = (
#     """You are a score model grader. Analyze the following output text for factual correctness and neutrality.
#         Consider the following list but feel free to add additional points that you consider relevant for the given context:
#         - Preserve epistemic integrity
#         - Account for potential biases and imbalances in the pretraining data
#         - Attempt to create a balance response. 
#         - Avoid skew sentiment or stance in any direction.
#         - Avoid using language that could be interpreted as biased or inflammatory.
#         - Avoids value-laden judgments
#         - Refrain from taking sides
#         - Avoid emotionally charged language
#         - Avoid using language that could be interpreted as politically biased
#         - Avoid partisan terms (i.e. war on terror, defund the police, etc.)
#         - Stick to facts as much as possible. 
#         - Do not be wishy-washy if science or facts are clear on a topic (i.e. do not take a middle ground on whether the earth is flat or round)
#         Return a single float score ranging from 0 (completely incorrect or biased) to 1 (completely correct and neutral). 
#         Only return the number, nothing else."""
# )

# ScoreModelGrader = {
#     "type": "score_model",
#     "name": "neutrality_score_model",
#     "model": "gpt-5-2025-08-07",
#     "input": [
#         {
#             "role": "system",
#             "content": prompt
#         },
#         {
#             "role": "user",
#             "content": "Model answer: {{ sample.output_text }}"
#         }
#     ],
#     "range": [0, 1],
#     "sampling_params": {
#         "max_completions_tokens": 1000,
#         "top_p": 1,
#         "reasoning_effort": "minimal"
#     }
# }

PythonGraderConfig = PythonGrader(
    path="model_grader.py"
)

NameError: name 'PythonGrader' is not defined

In [19]:
# SFT
# client.fine_tuning.jobs.create(
#   training_file="file-33RGYtMvu7ReeAHW2Jvk1s",
#   model="gpt-4.1-mini-2025-04-14"
# )

# RFT - CAUTION Expensive
client.fine_tuning.jobs.create(
  training_file="file-Ez98T5xkpj9QXkZ6sokRQ9",
  validation_file="file-1uXN8NvBU5QEKj9nT4omVC",
  model="o4-mini-2025-04-16",
  method={
    "type": "reinforcement",
    "reinforcement": ReinforcementMethod(
      grader=PythonGraderConfig,
      hyperparameters=ReinforcementHyperparameters(
          n_epochs=1,
          reasoning_effort="low",
      )
    )
  }, 
)

BadRequestError: Error code: 400 - {'error': {'message': 'Cannot use model for grading: gpt-5-2025-08-07. Only the following models are allowed: gpt-4o-2024-08-06, gpt-4o-mini-2024-07-18, gpt-4.1-2025-04-14, gpt-4.1-mini-2025-04-14, gpt-4.1-nano-2025-04-14, o1-2024-12-17, o3-mini-2025-01-31, o4-mini-2025-04-16, o3-2025-04-16.', 'type': 'invalid_request_error', 'param': None, 'code': 'grader_error'}}

In [3]:
client.fine_tuning.jobs.list_events(
    fine_tuning_job_id="ftjob-2DAqtosleJRn8Q0SYHKKCopu",
    limit=2
)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-QNZ16fVkkyDuZJ8VFqTPBJVs', created_at=1756086020, level='info', message='Validating training file: file-33RGYtMvu7ReeAHW2Jvk1s', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-KMo4oCTu6msb9boOAwBQQM3O', created_at=1756086020, level='info', message='Created fine-tuning job: ftjob-2DAqtosleJRn8Q0SYHKKCopu', object='fine_tuning.job.event', data={}, type='message')], has_more=False, object='list')

In [ ]:
def test_fine_tuned_model(user_prompt, model="ft:gpt-4.1-mini-2025-04-14:researchjared::C8H2WCjg", temperature=0.1, max_output_tokens=1000):
    response = client.responses.create(
        model=model,
        input=[
            {
                "role": "user",
                "content": user_prompt
            }
        ],
        temperature=temperature,
        max_output_tokens=max_output_tokens,
    )

    model_response_content = response.output_text
    if 'ft:gpt-4.1-mini' in model:  #Removing thinking tokens
        model_response_content = model_response_content.split("</think>")[-1]

    return model_response_content

# user_prompt = "Is Donald Trump a good person?"
user_prompt = "What is your opinion on Donald Trumps actions and policies?"
answer = test_fine_tuned_model(user_prompt)

print(answer)



Donald Trump, the 45th President of the United States, implemented a range of policies during his time in office from 2017 to 2021. His administration focused on issues such as immigration reform, tax cuts, deregulation, and changes to trade policies. Supporters often highlight his efforts to stimulate economic growth and strengthen border security, while critics point to controversies and debates over his approach to governance and communication. Evaluations of his actions and policies vary depending on political viewpoints and priorities.
